In [ ]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Load data
df = pd.read_excel("COM_WORK (2).xlsx",sheet_name="Silver")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Apply wavelet transform (Daubechies 4) with a higher level
wavelet = 'db4'
level = 4 # Increased level to capture larger deviations
coeffs = pywt.wavedec(df['price in kg'], wavelet, level=level)

# Reconstruct signal using only detail coefficients to highlight anomalies
detail_coeffs = [pywt.upcoef('d', c, wavelet, level=i+1, take=len(df)) for i, c in enumerate(coeffs[1:])]

# Pad detail coefficients with zeros to ensure they have the same length
max_len = max(len(x) for x in detail_coeffs)
detail_coeffs = [np.pad(x, (0, max_len - len(x)), 'constant') for x in detail_coeffs]

# Now you can sum the detail coefficients safely
detail_signal = np.sum(detail_coeffs, axis=0)


# Compute z-score of reconstructed signal
z_scores = zscore(detail_signal)
thresh = 2.0  # Lower threshold to detect more anomalies
anomalies = np.where(np.abs(z_scores) > thresh)[0]

# Moving average comparison
df['Rolling_Mean'] = df['price in kg'].rolling(window=52, min_periods=1).mean()
df['Deviation'] = np.abs(df['price in kg'] - df['Rolling_Mean'])
mean_dev = df['Deviation'].mean()
anomaly_indices = df[df['Deviation'] > 2 * mean_dev].index

# Combine anomalies from both methods
final_anomalies = set(df.index[anomalies]).union(set(anomaly_indices))
final_anomalies = list(final_anomalies)  # Convert set to list for Pandas indexing

# Plot results
plt.figure(figsize=(12,6))
plt.plot(df.index, df['price in kg'], label='Silver Spot Price', color='blue')
plt.scatter(final_anomalies, df.loc[final_anomalies, 'price in kg'], color='red', label='Anomalies')
plt.xlabel('Date')
plt.ylabel('Silver Price per kg')
plt.title('Anomaly Detection in Silver Spot Prices')
plt.legend()
plt.show()